# Option PNL Exercise - Simple

Whitney Rueckl (Fall 2019)

**Objective:** The goal of this exercise is to calculate the delta-hedged PNL of a short option position. We'll Calculate the PNL of selling this option starting on 1/2/2018 and then delta-hedging it through its expiration on 12/31/2018. Note: we will assume that the risk-free rate and dividend yield are both zero.

**Data:** The data file `option_pnl.csv` contains the price data for a particular option that traded in the market. 

**Questions to answer**:
1. What was the implied volatility of the option at the time of execution?
1. What was the realized volatility of the underlying over the life of the delta-hedged option position?
1. Based on the difference between implied volatility and realized volatility, would you expect to make or lose money?
1. What is the cumulative PNL for holding this delta-hedged option position?
1. How does the actual PNL compare with the PNL that would be predicted by the vega at the time of execution?  What could explain the discrepancies?

In [11]:
import numpy as np
import pandas as pd
import os as os

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

from py_vollib.black_scholes_merton import black_scholes_merton
from py_vollib.black_scholes_merton.greeks.analytical import delta
from py_vollib.black_scholes_merton.greeks.analytical import vega
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility

pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings('ignore')

# Supress scientific notation for large numbers
pd.options.display.float_format = '{:.2f}'.format


In [5]:
os.getcwd()

'C:\\Users\\whitn\\OneDrive\\Documents\\School\\Python Projects\\01_option_pnl'

In [8]:
df_option_data = pd.read_csv(str(os.getcwd()) + '\\option_pnl.csv')
# Read option data

df_option_data.head()

,underlying,buy_sell,expiration,cp,strike,trade_date,d2x,t2x,upx,option_price,volatility
0,SPY,-1,2018-12-31,p,269,2018-01-02,251,1.00,269.68,13.88,0.13
1,SPY,-1,2018-12-31,p,269,2018-01-03,250,0.99,271.59,12.97,0.13
2,SPY,-1,2018-12-31,p,269,2018-01-04,249,0.99,272.70,12.61,0.13
3,SPY,-1,2018-12-31,p,269,2018-01-05,248,0.98,274.45,12.08,0.14
4,SPY,-1,2018-12-31,p,269,2018-01-08,247,0.98,275.11,11.85,0.14


In [17]:
# plot underlying price
fig = px.line(df_option_data, x='trade_date', y='upx', title = "SPY Price")
fig.show()

### 1. What is the the implied volatility of the option at the time of execution?

In [19]:
opt_px = df_option_data['option_price'][0]
spot_px = df_option_data['upx'][0]
strike_px = df_option_data['strike'][0]

In [20]:
implied_vol = implied_volatility(price = opt_px, S = spot_px, K = strike_px, t = 251/252, r = 0, q = 0, flag = 'p')
implied_vol = np.round(implied_vol, 4)
print("The implied volatility (pricing volatility) is:")
implied_vol

The implied volatility (pricing volatility) is:


0.1327

#### The implied volatility (pricing volatility) is: 0.1327

### 2. Realized volatility of the underlying over the life of the delta-hedged option position?

In [22]:
# Calculate daily returns of underlying
df_option_data['ret'] = df_option_data['upx'] / df_option_data['upx'].shift(1) - 1
df_option_data.head()

,underlying,buy_sell,expiration,cp,strike,trade_date,d2x,t2x,upx,option_price,volatility,ret
0,SPY,-1,2018-12-31,p,269,2018-01-02,251,1.00,269.68,13.88,0.13,NaN
1,SPY,-1,2018-12-31,p,269,2018-01-03,250,0.99,271.59,12.97,0.13,0.01
2,SPY,-1,2018-12-31,p,269,2018-01-04,249,0.99,272.70,12.61,0.13,0.00
3,SPY,-1,2018-12-31,p,269,2018-01-05,248,0.98,274.45,12.08,0.14,0.01
4,SPY,-1,2018-12-31,p,269,2018-01-08,247,0.98,275.11,11.85,0.14,0.00


In [24]:
# plot underlying price
fig = px.line(df_option_data, x='trade_date', y='ret', title = "SPY Daily Returns")
fig.show()

In [33]:
# Calculate standard deviation of daily returns of the underlying (realized volatility)
real_vol = df_option_data['ret'].std() * np.sqrt(252)
df_option_data['realized_vol'] = df_option_data['ret'].rolling(251).std() * np.sqrt(252)
print("Realized volatility (pricing volatility) is:")
np.round(real_vol, 6)
#df_option_data['ret'].std() * np.sqrt(252)
#df_option_data['ret'].std()

Realized volatility (pricing volatility) is:


0.1719

#### Realized volatility (pricing volatility) is: 0.1719. 

### 3. This is greater than the implied volatility, and given we have a short position we lost money. <br>

#### Pricing volatility is greater than implied volatility. Given we have a short position we lost money on the option. (On a short position, if realized vol = implied vol, break even. If realized vol > implied then lose money)

### 4. What is the cumulative profit-and-loss on the delta hedged position?

In [38]:
df_path = \
    (
    pd.DataFrame(
        {'underlying':'SPY',
         'cp':'p',
         'strike':269,
         'volatility': df_option_data['volatility'],
         'upx':df_option_data['upx'], 
         'd2x':list(range(251, -1, -1)),
         'buy_sell':-1,
        }       
    )
    .assign(t2x = lambda df: df.d2x / 252)
    )
df_path

# t2x = time to expiration
# d2x = trading days to expiration

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x
0,SPY,p,269,0.13,269.68,251,-1,1.00
1,SPY,p,269,0.13,271.59,250,-1,0.99
2,SPY,p,269,0.13,272.70,249,-1,0.99
3,SPY,p,269,0.14,274.45,248,-1,0.98
4,SPY,p,269,0.14,275.11,247,-1,0.98
...,...,...,...,...,...,...,...,...
247,SPY,p,269,0.59,233.50,4,-1,0.02
248,SPY,p,269,0.43,246.64,3,-1,0.01
249,SPY,p,269,0.40,248.72,2,-1,0.01
250,SPY,p,269,0.52,247.81,1,-1,0.00


In [39]:
# Function to calculate price of the option (vectorized)
def bsm_px(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0
    volatility = row['volatility']
    q = 0
    px = black_scholes_merton(cp, upx, strike, t2x, rf, volatility, q)
    px = np.round(px, 2)
    return(px)

In [40]:
def bsm_delta(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0
    volatility = row['volatility']
    q = 0
    if t2x == 0:
        return(0)
    diff = delta(cp, upx, strike, t2x, rf, volatility, q)
    diff = np.round(diff, 3)
    return(diff)

In [41]:
def bsm_vega(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0
    volatility = row['volatility']
    q = 0
    if t2x == 0:
        return(0)
    vga = vega(cp, upx, strike, t2x, rf, volatility, q)
    vga = np.round(vga, 3)
    return(vga)

In [42]:
df_path['option_price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option_price
0,SPY,p,269,0.13,269.68,251,-1,1.00,13.88
1,SPY,p,269,0.13,271.59,250,-1,0.99,12.98
2,SPY,p,269,0.13,272.70,249,-1,0.99,12.61
3,SPY,p,269,0.14,274.45,248,-1,0.98,12.08
4,SPY,p,269,0.14,275.11,247,-1,0.98,11.85
...,...,...,...,...,...,...,...,...,...
247,SPY,p,269,0.59,233.50,4,-1,0.02,35.70
248,SPY,p,269,0.43,246.64,3,-1,0.01,22.51
249,SPY,p,269,0.40,248.72,2,-1,0.01,20.32
250,SPY,p,269,0.52,247.81,1,-1,0.00,21.21


In [43]:
# Calculate delta
df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option_price,delta
0,SPY,p,269,0.13,269.68,251,-1,1.00,13.88,-0.47
1,SPY,p,269,0.13,271.59,250,-1,0.99,12.98,-0.45
2,SPY,p,269,0.13,272.70,249,-1,0.99,12.61,-0.43
3,SPY,p,269,0.14,274.45,248,-1,0.98,12.08,-0.41
4,SPY,p,269,0.14,275.11,247,-1,0.98,11.85,-0.41
...,...,...,...,...,...,...,...,...,...,...
247,SPY,p,269,0.59,233.50,4,-1,0.02,35.70,-0.97
248,SPY,p,269,0.43,246.64,3,-1,0.01,22.51,-0.97
249,SPY,p,269,0.40,248.72,2,-1,0.01,20.32,-0.98
250,SPY,p,269,0.52,247.81,1,-1,0.00,21.21,-0.99


In [44]:
# Calculate the PNL from the option (change in the option price)
df_path['option_pnl'] = df_path['buy_sell'] * df_path['option_price'].diff()
df_path
df_path['option_pnl'].sum()

-5.039999999999997

#### The pnl on the position is a loss  of 5.04 dollars

In [48]:
df_path[1:].head(10)

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option_price,delta,option_pnl
1,SPY,p,269,0.13,271.59,250,-1,0.99,12.98,-0.45,0.90
2,SPY,p,269,0.13,272.70,249,-1,0.99,12.61,-0.43,0.37
3,SPY,p,269,0.14,274.45,248,-1,0.98,12.08,-0.41,0.53
4,SPY,p,269,0.14,275.11,247,-1,0.98,11.85,-0.41,0.23
5,SPY,p,269,0.14,275.62,246,-1,0.98,11.73,-0.40,0.12
6,SPY,p,269,0.14,275.40,245,-1,0.97,11.72,-0.41,0.01
7,SPY,p,269,0.14,277.37,244,-1,0.97,11.03,-0.39,0.69
8,SPY,p,269,0.14,279.35,243,-1,0.96,10.42,-0.37,0.61
9,SPY,p,269,0.14,278.71,242,-1,0.96,11.00,-0.37,-0.58
10,SPY,p,269,0.15,280.97,241,-1,0.96,10.35,-0.35,0.65


In [45]:
fig = px.line(df_path, x='d2x', y='option_pnl', title = "Daily Profit and Loss")
fig.show()

In [14]:
# Calculate the PNL from the delta hedge
# *my note*: delta hedge pnl = -buy_sell*(delta from (t-1))*(chg in price of underlying)
df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff() 
df_path
print("The delta hedge pnl is:")
cum_delta_hedge_pnl = df_path['delta_hedge_pnl'].sum()
cum_delta_hedge_pnl

The delta hedge pnl is:


1.2570030000000187

#### The delta hedge pnl is a gain of: 1.257 dollars

In [15]:
# Calculate the total PNL
df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
df_path
df_path['total_pnl'].sum()

-3.782996999999989

In [16]:
print("The total pnl is:")
df_path['total_pnl'].mean()

The total pnl is:


-0.015071701195219063

#### Total pnl is -3.78 and the average pnl is -0.015

### 5. How does the actual PNL compare with the PNL that would be predicted by the vega at the time of execution? What could explain the discrepancies?

In [17]:
# this returns vega divided by 100
vega = -bsm_vega(df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].iloc[0,:])
print("Vega is:")
#vega = vega*100
vega

Vega is:


-1.07

#### Vega is: -1.07

In [23]:
# Using the identity
x = -vega * (implied_vol - real_vol) * 100
x

-4.19435881127016

In [24]:
# Vega gives the change in the option price given a 1% change in the volatility of the underlying
# using the calculated vega of -1.07:
initial_option_price = df_option_data['option_price'].iloc[0]
adjusted_option_price = initial_option_price + ((implied_vol - real_vol)*vega*100)
adjusted_option_price

18.07935881127016

In [25]:
# the pnl on the option is the difference between the price we sold the option for and how much it is worth at expiration
option_pnl_predicted_by_vega = (initial_option_price - adjusted_option_price)
option_pnl_predicted_by_vega

#this gives the same value as: -vega * (implied_vol - real_vol) * 100

-4.1943588112701615

In [26]:
adjusted_total_pnl = df_path['delta_hedge_pnl'].sum() + option_pnl_predicted_by_vega
print("Using the option pnl predicted by vega, the total pnl is:")
adjusted_total_pnl

Using the option pnl predicted by vega, the total pnl is:


-2.937355811270143

#### Using the option pnl predicted by vega, the total pnl is: -2.94

#### Answer: Using vega, I would have predicted the option pnl to be -4.124, and total pnl to be -2.937, while our actual option pnl is -5.04 and the total pnl is -3.783. The pnl predicted by vega is less negative than the actual.  The driver behind the discrepancy is that we are not re-calculating vega over the life of the option. 